In [ ]:
import OffscreenRenderer as ogl, video_writer
import pickle, gzip, numpy as np

In [ ]:
width, height, P, N, C, modParams, camParams = pickle.load(gzip.open('data/render_data.pkl.gz'))
width, height = 2 * np.array([width, height]) # Render at 2x resolution
# C = np.array([0.5, 0.7, 1.0]) # override input colors with a constant color
camParams[0] += np.array(camParams[2]) # View from slightly above

In [ ]:
mrender = ogl.MeshRenderer(width, height)
mrender.setMesh(P, None, N, C)

mrender.lookAt(*camParams)
mrender.modelMatrix(*modParams)
mrender.perspective(50, width / height, 0.1, 2000)

mrender.meshes[0].alpha = 0.5
mrender.meshes[0].lineWidth = 1.5
mrender.meshes[0].shininess = 100.0
mrender.specularIntensity[:] = 2.0

In [ ]:
# Render and display image inline
mrender.render()
mrender.image().resize((width//2, height//2))

## Write an animation using MeshRenderer's animation callback

Default settings (H264, 30fps, full size)

In [ ]:
nframes = 360
def cb(r, i): r.orbitedLookAt(*camParams, camParams[-1], 2  * np.pi / nframes * i)
mrender.renderAnimation('orbit.mp4', nframes, cb)

HEVC output, 60fps, downscaled

In [ ]:
nframes = 360
mrender.renderAnimation('orbit_scaled_hevc.mp4', nframes, cb, framerate=60,
                        outWidth=width//2, outHeight=height//2, codec=video_writer.Codec.HEVC)

PNG image sequence with transparent background (slow, large files :()

In [ ]:
mrender.transparentBackground = True
mrender.renderAnimation('orbit_image_seq', nframes, cb,
                        outWidth=width//2, outHeight=height//2, codec=video_writer.Codec.ImgSeq)

## Write an animation using the lower-level MeshRendererVideoWriter class

In [ ]:
vw = video_writer.MeshRendererVideoWriter('orbit_lowlevel.mp4', mrender, outWidth=width//2, outHeight=height//2)
nframes = 360
for angle in np.linspace(0, 2 * np.pi, nframes):
    mrender.orbitedLookAt(*camParams, camParams[-1], angle)
    vw.writeFrame()
del vw